*Щоб код запрацював виконай це у терміналі:*  
pip install pandas sqlalchemy psycopg2


In [217]:
import pandas as pd
import psycopg2 as ps
from datetime import datetime, timedelta


## Data export from DB to Dataframe

In [218]:
connection = ps.connect(
    dbname="statistic",
    user="postgres",
    password="123",
    host="localhost",
    port="5432"
)

In [219]:
cursor = connection.cursor()

In [220]:
def clear_names(data):
    clear_data = []
    for i in data:
        clear_data.append(str(i)[2: -3])
    return clear_data
    

In [221]:
def dataframes(tables):
    df = []
    for table in tables:
        cursor.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table}' ORDER BY ordinal_position")
        names = clear_names(cursor.fetchall())
        cursor.execute(f"SELECT * FROM {table}")
        df.append(pd.DataFrame(cursor.fetchall(), columns = names))
    return df

In [222]:
tables = ['issues', 'changelogs']
df = dataframes(tables)
issues, changelogs = df[0], df[1]

cursor.close()

# 3 task

In [223]:
issues.head(5)

,issue_key,project_key,issue_type,assignee_key,created_at
0,PYTHON-11,PYTHON,Bug,mike,2009-04-15 11:18:46
1,PYTHON-1,PYTHON,Improvement,mike,2009-04-09 09:38:38
2,PYTHON-10,PYTHON,Task,mike,2009-04-13 16:34:03
3,PYTHON-100,PYTHON,Improvement,mike,2010-02-11 10:23:30
4,PYTHON-1000,PYTHON,Bug,behackett,2015-10-06 08:57:47


In [224]:
changelogs.head(5)

,issue_key,author_key,from_status,to_status,created_at
0,PYTHON-109,mike,Open,Closed,2010-03-10 12:13:11
1,PYTHON-108,mike,Open,Closed,2010-08-19 15:58:33
2,PYTHON-107,mike,Open,Closed,2010-03-02 12:08:58
3,PYTHON-106,mike,Open,Closed,2010-05-10 17:41:21
4,PYTHON-105,mike,Open,Closed,2010-02-22 15:45:32


In [225]:
issues['assignee_key'] = pd.to_datetime(issues['created_at'])
changelogs['created_at'] = pd.to_datetime(changelogs['created_at'])


In [226]:
# Об'єднання датафреймів в 1 спільний
data = pd.merge(changelogs, issues[['issue_key', 'created_at']], on='issue_key', how='inner', suffixes=('_finish', '_start'))


In [227]:
data = data[['issue_key', 'author_key', 'from_status', 'to_status', 'created_at_start', 'created_at_finish']]
data

,issue_key,author_key,from_status,to_status,created_at_start,created_at_finish
0,PYTHON-109,mike,Open,Closed,2010-03-05 18:51:36,2010-03-10 12:13:11
1,PYTHON-108,mike,Open,Closed,2010-03-05 14:28:32,2010-08-19 15:58:33
2,PYTHON-107,mike,Open,Closed,2010-03-01 04:07:10,2010-03-02 12:08:58
3,PYTHON-106,mike,Open,Closed,2010-02-28 19:54:09,2010-05-10 17:41:21
4,PYTHON-105,mike,Open,Closed,2010-02-19 22:54:44,2010-02-22 15:45:32
...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,Resolved,2009-04-09 09:40:32,2011-07-12 02:47:50
8915,PYTHON-4,behackett,Resolved,Closed,2009-04-09 09:40:32,2011-08-05 23:04:33
8916,PYTHON-3,mike,Open,Closed,2009-04-09 09:39:56,2009-10-26 12:08:50
8917,PYTHON-2,mike,Open,Closed,2009-04-09 09:39:06,2009-07-20 17:19:39


In [228]:
data.dropna(inplace=True)

Отже, логіка наступна:

Є times(data), яка фільтрує дані, які мали from_status - In progress. Після фільтрації, створюється колонка times, яка показує скільки робочого часу задача була в статусі за допомогою векторизаціїї та функції calc().

1. Calc отримує рядок даних, розбиває часовий ряд на: перший день, центральні дні, останній день.
2. Перший і останній дні розраховуються за рахунок того, що береться початок доби, або кінець доби і створюється штучний часовий ряд, який потім попадає у функцію first_last_day().
3. У цій функції створюється маска робочого дня, яка накладається на масив секунд з data_range, для фільтрації робочого часу. Потім рахується к-сть елементів = к-сть секунд.

4. Central_days: береться з 2-го дня до передостаннього, створюється фільтр та рахуємо к-сть робочих днів, які переводимо в секунди.
5. Calc: повертає суму першого\останнього дня та центральних днів.
6. Times: Повертаються дані з уже готовим стовбцем робочого часу задачі.


In [1]:
def first_last_day(start_time, end_time):
    
    time_range = pd.date_range(start=start_time, end=end_time, freq='S')

    # Фільтрація за днями та годинами
    workdays = time_range[(time_range.hour >= 10) & (time_range.hour < 20)]

    # Обчислення сумарного часу(секунди)
    total_seconds = len(workdays)

    return total_seconds

In [230]:
def central_days(start_time, end_time):
    # Наступний день після стартового
    next_day = (start_time + pd.Timedelta(days=1)).replace(hour=0, minute=0, second=0)
    
    # Попередній день перед кінцевою датою
    prev_day = (end_time - pd.Timedelta(days=1)).replace(hour=23, minute=59, second=59)
    
    if next_day > prev_day:
        return 0

    # Обчислення кількості робочих днів між next_day та prev_day
    central_days = pd.date_range(start=next_day, end=prev_day, freq='B')
    central_days_seconds = len(central_days) * 10 * 3600  # 10 годин робочого часу на день

    return central_days_seconds

In [231]:
def calc(start_time, end_time):
    # Рахуємо перший день
    first_day = first_last_day(start_time, start_time.replace(hour=23, minute=59, second=59))
    
    # Рахуємо останній день
    last_day = first_last_day(end_time.replace(hour=0, minute=0, second=0), end_time)
    
    # Рахуємо середні дні
    central = central_days(start_time, end_time)
    
    # Повертаємо суму всіх розрахунків
    total_seconds = first_day + last_day + central
    
    return total_seconds

In [232]:
def times(data):
    # Створення маски для фільтрації
    mask = (data['from_status'] == 'In Progress') & (data['to_status'] != 'In Progress')
    
    # Застосовуємо calculate_work_seconds до відфільтрованих даних та заповнюємо решту нулями
    data['times'] = 0
    data.loc[mask, 'times'] = data.loc[mask].apply(
        lambda row: calc(row['created_at_start'], row['created_at_finish']),
        axis=1
    )
    
    return data

In [233]:
data = times(data)

In [234]:
data

,issue_key,author_key,from_status,to_status,created_at_start,created_at_finish,times
0,PYTHON-109,mike,Open,Closed,2010-03-05 18:51:36,2010-03-10 12:13:11,0
1,PYTHON-108,mike,Open,Closed,2010-03-05 14:28:32,2010-08-19 15:58:33,0
2,PYTHON-107,mike,Open,Closed,2010-03-01 04:07:10,2010-03-02 12:08:58,0
3,PYTHON-106,mike,Open,Closed,2010-02-28 19:54:09,2010-05-10 17:41:21,0
4,PYTHON-105,mike,Open,Closed,2010-02-19 22:54:44,2010-02-22 15:45:32,0
...,...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,Resolved,2009-04-09 09:40:32,2011-07-12 02:47:50,0
8915,PYTHON-4,behackett,Resolved,Closed,2009-04-09 09:40:32,2011-08-05 23:04:33,0
8916,PYTHON-3,mike,Open,Closed,2009-04-09 09:39:56,2009-10-26 12:08:50,0
8917,PYTHON-2,mike,Open,Closed,2009-04-09 09:39:06,2009-07-20 17:19:39,0


# Task_4

In [235]:
# фільтруємо дані, щоб статус був лише "In progress"
data_prog = data[data['times'] != 0]

## Функції

1. Почнемо з combine(), яка лише комбінує дані(не знаю навіщо, так легше було контролювати процес)
2. func_of_find(): функція для пошуку перетинів між даними. Якщо просто: береться на вхід рядок, фільтрується за author_key та шукає перетини часу по всіх даних. Якщо перетини є, то вони додаються у форматі {index_row: [index1]}

In [236]:
def time_cond(start_time, end_time):

    if (end_time - start_time).total_seconds() == 0:

        return 0
    
    elif (start_time.strftime('%y-%m-%d') == end_time.strftime('%y-%m-%d')):

        return first_last_day(start_time, end_time)
    
    else:
        return calc(start_time, end_time)

In [237]:
def pairs(pairs_data, df):

    list = []

    for key, values in pairs_data.items():

        if len(values) == 0:
            
            list.append(time_cond(df.iloc[key]['created_at_start'], df.iloc[key]['created_at_finish']))

        else:
            a_start = df.iloc[key]['created_at_start']
            a_finish = df.iloc[key]['created_at_finish']

            total = 0      

            for pair in values:
                b_start = df.loc[pair]['created_at_start']
                b_finish = df.loc[pair]['created_at_finish']

                ab_max = max(a_start, b_start)
                ab_min = min(a_finish, b_finish)

                if (ab_max <= ab_min):
                        
                        comp_1 = time_cond(a_start, ab_max)
                        comp_2 = time_cond(ab_max, ab_min) / 2 
                        comp_3 = time_cond(ab_min, a_finish)
                        
                        a_time = int(comp_1 + comp_2 + comp_3)
                        
                else:
                    a_time = (a_finish - a_start).total_seconds()
                
                total += a_time

            list.append(total)        

    return list

In [238]:
def func_of_find(data):
    
    overlap_dict = {}

    for i in range(len(data)):

        name = data.iloc[i]['author_key']
        start = data.iloc[i]['created_at_start']
        end = data.iloc[i]['created_at_finish']

        mask = ((data['author_key'] == name) & (data['created_at_start'] <= end) & (data['created_at_finish'] >= start))

        overlapping_indices = data.index[mask].tolist()

        if i in overlapping_indices:
            overlapping_indices.remove(i)

        overlap_dict[i] = overlapping_indices
    return overlap_dict

In [239]:
def combine(data):
    return pairs(func_of_find(data), data)

## Приклад з тестового

In [240]:
data_2 = {
    'issue_key': ['A', 'B'],
    'author_key': ['Jason', 'Jason'],
    'from_status': ['Open', 'Open'],
    'to_status': ['Closed', 'Closed'],
    'created_at_start': ['2024-01-19 10:00:00', '2024-01-22 10:00:00'],
    'created_at_finish': ['2024-01-23 14:00:00', '2024-01-23 20:00:00']
}


df_2 = pd.DataFrame(data_2)


df_2['created_at_start'] = pd.to_datetime(df_2['created_at_start'])
df_2['created_at_finish'] = pd.to_datetime(df_2['created_at_finish'])

df_2['working_duration'] = combine(df_2)

In [241]:
df_2

,issue_key,author_key,from_status,to_status,created_at_start,created_at_finish,working_duration
0,A,Jason,Open,Closed,2024-01-19 10:00:00,2024-01-23 14:00:00,61201
1,B,Jason,Open,Closed,2024-01-22 10:00:00,2024-01-23 20:00:00,46800


## Результати

In [242]:
data_prog

,issue_key,author_key,from_status,to_status,created_at_start,created_at_finish,times
17,PYTHON-3049,behackett,In Progress,In Code Review,2021-12-14 21:53:57,2021-12-17 18:01:11,100872
29,PYTHON-3042,behackett,In Progress,In Code Review,2021-12-13 16:16:23,2021-12-14 22:56:05,49417
44,PYTHON-3034,JIRAUSER1253561,In Progress,In Code Review,2021-12-03 21:18:26,2021-12-03 22:07:05,36000
48,PYTHON-3033,shane.harvey,In Progress,In Code Review,2021-12-03 12:56:55,2021-12-06 18:41:35,56681
62,PYTHON-3027,shane.harvey,In Progress,In Code Review,2021-11-30 23:53:41,2021-12-02 20:59:26,72000
...,...,...,...,...,...,...,...
8810,PYTHON-89,mike,In Progress,Closed,2010-01-18 09:43:55,2010-01-19 11:37:26,41847
8825,PYTHON-78,mike,In Progress,Closed,2009-12-09 15:51:15,2010-01-05 11:59:34,670100
8885,PYTHON-29,mike,In Progress,Closed,2009-07-09 15:41:43,2009-07-15 12:41:31,133189
8903,PYTHON-12,mike,In Progress,Closed,2009-04-15 13:12:44,2009-04-15 13:59:02,38779


In [243]:
data_prog['working_duration'] = combine(data_prog)

C:\Users\MSI\AppData\Local\Temp\ipykernel_11612\951323125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_prog['working_duration'] = combine(data_prog)


In [244]:
data_prog

,issue_key,author_key,from_status,to_status,created_at_start,created_at_finish,times,working_duration
17,PYTHON-3049,behackett,In Progress,In Code Review,2021-12-14 21:53:57,2021-12-17 18:01:11,100872,151308
29,PYTHON-3042,behackett,In Progress,In Code Review,2021-12-13 16:16:23,2021-12-14 22:56:05,49417,122171
44,PYTHON-3034,JIRAUSER1253561,In Progress,In Code Review,2021-12-03 21:18:26,2021-12-03 22:07:05,36000,0
48,PYTHON-3033,shane.harvey,In Progress,In Code Review,2021-12-03 12:56:55,2021-12-06 18:41:35,56681,363388
62,PYTHON-3027,shane.harvey,In Progress,In Code Review,2021-11-30 23:53:41,2021-12-02 20:59:26,72000,396000
...,...,...,...,...,...,...,...,...
8810,PYTHON-89,mike,In Progress,Closed,2010-01-18 09:43:55,2010-01-19 11:37:26,41847,20923
8825,PYTHON-78,mike,In Progress,Closed,2009-12-09 15:51:15,2010-01-05 11:59:34,670100,335050
8885,PYTHON-29,mike,In Progress,Closed,2009-07-09 15:41:43,2009-07-15 12:41:31,133189,66594
8903,PYTHON-12,mike,In Progress,Closed,2009-04-15 13:12:44,2009-04-15 13:59:02,38779,1389


# Task 5

In [245]:
data_5 = pd.merge(issues[['issue_key', 'issue_type']], data_prog[['issue_key', 'author_key', 'working_duration']], on='issue_key', how='inner')

In [252]:
data_5.head(5)

,issue_key,issue_type,author_key,working_duration
0,PYTHON-1003,Bug,jesse,3538
1,PYTHON-1006,New Feature,behackett,29214
2,PYTHON-1009,Improvement,behackett,1370209
3,PYTHON-1010,Bug,anna.herlihy@10gen.com,46732
4,PYTHON-1011,Bug,behackett,715059


In [256]:
def cost(row):
    if 'JIRAUSER' in row['author_key']:
        return row['working_duration'] / 3600 * 1
    else:
        return row['working_duration'] / 3600 * 2

In [257]:
data_5['cost'] = data_5.apply(cost, axis=1)


In [258]:
data_5

,issue_key,issue_type,author_key,working_duration,cost
0,PYTHON-1003,Bug,jesse,3538,1.965556
1,PYTHON-1006,New Feature,behackett,29214,16.230000
2,PYTHON-1009,Improvement,behackett,1370209,761.227222
3,PYTHON-1010,Bug,anna.herlihy@10gen.com,46732,25.962222
4,PYTHON-1011,Bug,behackett,715059,397.255000
...,...,...,...,...,...
1344,PYTHON-990,New Feature,anna.herlihy@10gen.com,383760,213.200000
1345,PYTHON-991,Bug,behackett,4076326,2264.625556
1346,PYTHON-996,Task,behackett,2416884,1342.713333
1347,PYTHON-997,Improvement,behackett,1378546,765.858889


In [259]:
grouped_data = data_5.groupby('issue_type')['cost'].sum().reset_index()

In [260]:
grouped_data

,issue_type,cost
0,Bug,1.585925e+07
1,Epic,4.408366e+06
2,Improvement,3.594475e+07
3,New Feature,1.667958e+07
4,Spec Change,8.164937e+05
5,Sub-task,1.343936e+04
6,Task,7.718516e+07
